# Etiquetado NER

El etiquetado NER buscar marcar los tokens que pertenecen a una misma entidad nombrada (persona, organización, lugar, etc.) que nos ayudará a realziar recuperación de información.

Implementamos un etiquetado NER simple en base al algoritmo de HMM.

In [1]:
from nltk.tag import hmm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from itertools import chain
from tqdm import tqdm

### Preparación del dataset

El dataset es un corpus con etiquetas NER y POS. Las etiquetas NER tienen un formato BIO para identificar el inicio de cadenas de entidades nombradas. Guardamos las cadenas con etiquetas NER y limpiamos el corpus. Asimismo, separamos en datos de entrenamiento y evaluación.

El corpus se obtuvo de <u>https://github.com/juand-r/entity-recognition-datasets</u>

In [2]:
#Abre el documento con corpus
text = open('NERSpanish.txt','r').read().strip().split('\n')

#Corpus NER
corpus_NER = []
#Corpus NER+POS
corpus_FULL = []
for sent in tqdm(text):
    #Guarda oraciones con POS+NER
    tagged_sent = []
    #Guarda oraciones con NER
    NER_sent = []
    for pack in sent.split():
        #Separa en token, POS, NER
        w, pos, ner = pack.split('|')
        #Guarda POS+NER
        tagged_sent.append((w,pos,ner))
        #Guarda NER
        NER_sent.append((w,ner))
    corpus_FULL.append(tagged_sent)
    corpus_NER.append(NER_sent)

#Elimina elementos vacíos del corpus NER
corpus = [sent for sent in corpus_NER if sent != []]

100%|██████████| 145189/145189 [00:07<00:00, 19099.16it/s]


Finalmente, obtenemos los datos de entrenamiento y evaluación.

In [3]:
#Dividimos en evaluación y entrenamiento
train_data, test_data = train_test_split(corpus, test_size=0.1)

print(len(train_data), len(test_data))

115519 12836


## Modelo oculto de Markov

### Entrenamiento

Tomamos los datasets de entrenamiento y generamos en modelo en base a la biblioteca de NLTK.

In [4]:
#Genera el modelo
HMM = hmm.HiddenMarkovModelTrainer()
#Entrena el modelo
NERtagger = HMM.train_supervised(train_data)

### Evaluación

Evaluamos el modelo con base en el dataset de evaluación.

In [5]:
#Etiquetas reales
Y = []
#Etiquetas predichas
Y_pred = []
for test_sent in tqdm(test_data):
    #Observaciones y emisiones
    x,y = zip(*test_sent)
    #Predicción
    y_pred = NERtagger.tag(list(x))
    
    #Guarda las etiquetas
    Y.append(list(y))
    Y_pred.append([tag[1] for tag in y_pred])

100%|██████████| 12836/12836 [03:01<00:00, 70.56it/s] 


Obtenemos el reporte de clasificación para cada etiqueta.

In [6]:
print(classification_report(list(chain(*Y)), list(chain(*Y_pred))))

/home/mijangos/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mijangos/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-LOC       0.00      0.00      0.00         6
      B-MISC       0.00      0.00      0.00         2
       B-ORG       0.00      0.00      0.00         0
       B-PER       0.00      0.00      0.00         6
       I-LOC       0.92      0.73      0.81     18080
      I-MISC       0.84      0.57      0.68      6932
       I-ORG       0.88      0.70      0.78      4015
       I-PER       0.92      0.75      0.82     11887
           O       0.97      1.00      0.98    309535

    accuracy                           0.96    350463
   macro avg       0.50      0.42      0.45    350463
weighted avg       0.96      0.96      0.96    350463



Podemos probar cómo funciona en un caso específico:

In [7]:
sent = 'Yo vivo en el Valle de México'
NERtagger.tag(sent.split())

[('Yo', 'O'),
 ('vivo', 'O'),
 ('en', 'O'),
 ('el', 'O'),
 ('Valle', 'I-LOC'),
 ('de', 'I-LOC'),
 ('México', 'I-LOC')]